In [1]:
import torch
import numpy as np
import time
import timeit


In [8]:
d = 200
X = np.random.standard_normal(d)
a = np.random.standard_normal(d)
diff = X - a
norm = np.linalg.norm(diff)

def method_1(diff, norm):
    diff_scaled = diff / norm ** 1.5
    return np.outer(diff_scaled, diff_scaled)

def method_2(diff, norm):
    return np.outer(diff, diff / norm ** 3)

def method_3(diff, norm):
    return np.outer(diff, diff) / norm ** 3

time_1 = timeit.timeit(lambda: method_1(diff, norm), number=1000)
time_2 = timeit.timeit(lambda: method_2(diff, norm), number=1000)
time_3 = timeit.timeit(lambda: method_3(diff, norm), number=1000)
time_4 = timeit.timeit(lambda: method_1(diff, norm), number=1000)
time_5 = timeit.timeit(lambda: method_2(diff, norm), number=1000)
time_6 = timeit.timeit(lambda: method_3(diff, norm), number=1000)

print(f"Method 1: {time_1}")
print(f"Method 2: {time_2}")
print(f"Method 3: {time_3}")
print(f"Method 1: {time_4}")
print(f"Method 2: {time_5}")
print(f"Method 3: {time_6}")

Method 1: 0.05295457500005796
Method 2: 0.04317028300010861
Method 3: 0.06615527199983262
Method 1: 0.040630676000091626
Method 2: 0.04155634700009614
Method 3: 0.06714793900027871


In [50]:
n = 20  # Number of vectors
d = 20  # Dimensionality of vectors
v = torch.randn(n, d)
w = torch.rand(n)


# Method 0
def method_0(v, w):
    n, d = v.shape
    A = torch.zeros(d, d)
    for i in range(n):
        add = w[i] * torch.outer(v[i], v[i])
        A += add
    return A


# Method 1
def method_1(v, w):
    outerProds = torch.einsum("nd,nb->ndb", v, v)
    A = torch.einsum("ndb,n->db", outerProds, w)
    return A


# Method 2
def method_2(v, w):
    vScaled = torch.einsum("nd,n->nd", v, torch.sqrt(w))
    A = torch.einsum("nd,nb->db", vScaled, vScaled)
    return A


# Method 3
def method_3(v, w):
    Y = torch.einsum("nd,n->nd", v, w)
    A = torch.einsum("nd,nb->db", Y, v)
    return A / n


# Method 4 (normalized)
def method_4(v, w):
    n = w.size(0)
    Y = torch.einsum("nd,n->nd", v, w / n)
    A = torch.einsum("nd,nb->db", Y, v)
    return A


# Method 5 (direct einsum)
def method_5(v, w):
    A = torch.einsum("nd,n,nb->db", v, w, v)
    return A / n


# Method 6 (direct einsum normalized)
def method_6(v, w):
    A = torch.einsum("nd,n,nb->db", v, w / n, v)
    return A


# Method 7 (matrix multiplication)
def method_7(v, w):
    Y = torch.matmul(torch.diag(w/n), v)
    A = torch.matmul(Y.T, v)
    return A


sm0 = method_0(v, w)
sm1 = method_1(v, w)
sm2 = method_2(v, w)
sm3 = method_3(v, w) * n
sm4 = method_4(v, w) * n
sm5 = method_5(v, w) * n
sm6 = method_6(v, w) * n
sm7 = method_7(v, w) * n

# test if same result
epsilon = 1e-6
print(torch.allclose(sm0, sm1, atol=epsilon))
print(torch.allclose(sm0, sm2, atol=epsilon))
print(torch.allclose(sm0, sm3, atol=epsilon))
print(torch.allclose(sm0, sm4, atol=epsilon))
print(torch.allclose(sm0, sm5, atol=epsilon))
print(torch.allclose(sm0, sm6, atol=epsilon))
print(torch.allclose(sm0, sm7, atol=epsilon))

True
True
True
True
True
True
True


In [63]:
# Benchmark
n = 20
d = 20
v = torch.randn(n, d)
w = torch.rand(n)

# time_0 = timeit.timeit('method_0(v, w)', globals=globals(), number=1000)
time_1 = timeit.timeit('method_1(v, w)', globals=globals(), number=1000)
time_2 = timeit.timeit('method_2(v, w)', globals=globals(), number=1000)
time_3 = timeit.timeit('method_3(v, w)', globals=globals(), number=1000)
time_4 = timeit.timeit('method_4(v, w)', globals=globals(), number=1000)
time_5 = timeit.timeit('method_5(v, w)', globals=globals(), number=1000)
time_6 = timeit.timeit('method_6(v, w)', globals=globals(), number=1000)
time_7 = timeit.timeit('method_7(v, w)', globals=globals(), number=1000)

# print(f"Method 0: {time_0}")
print(f"Method 1: {time_1}")
print(f"Method 2: {time_2}")
print(f"Method 3: {time_3}")
print(f"Method 4: {time_4}")
print(f"Method 5: {time_5}")
print(f"Method 6: {time_6}")
print(f"Method 7: {time_7}")



Method 1: 0.017966649000300094
Method 2: 0.01636508999945363
Method 3: 0.01826516199798789
Method 4: 0.01874012700136518
Method 5: 0.016985008005576674
Method 6: 0.016278536997560877
Method 7: 0.010616023006150499


In [66]:
import numpy as np
import timeit

# Create a random matrix X
n = 10
d = 10
X = np.random.standard_normal((n, d))

# Function to compute covariance matrix using np.einsum with division outside
def einsum_method_1(X, n):
    return np.einsum("ni,nj->ij", X, X) / n

# Function to compute covariance matrix using np.einsum with division inside
def einsum_method_2(X, n):
    return np.einsum("ni,nj->ij", X / n, X)

# Test for np.einsum with division outside
time_einsum_1 = timeit.timeit('einsum_method_1(X, n)', globals=globals(), number=1000)

# Test for np.einsum with division inside
time_einsum_2 = timeit.timeit('einsum_method_2(X, n)', globals=globals(), number=1000)

print(f"Time for np.einsum with division outside: {time_einsum_1:.6f} seconds")
print(f"Time for np.einsum with division inside: {time_einsum_2:.6f} seconds")


Time for np.einsum with division outside: 0.014524 seconds
Time for np.einsum with division inside: 0.006842 seconds


## Test pytorch vs numpy speed on CPU

In [15]:
# Create random matrices
matrix_size = 1000
np_matrix1 = np.random.standard_normal(matrix_size, matrix_size)
np_matrix2 = np.random.standard_normal(matrix_size, matrix_size)
torch_matrix1 = torch.tensor(np_matrix1)
torch_matrix2 = torch.tensor(np_matrix2)

# PyTorch matrix multiplication
start = time.time()
torch_result = torch.matmul(torch_matrix1, torch_matrix2)
end = time.time()
print(f"PyTorch time: {end - start} seconds")

# NumPy matrix multiplication
start = time.time()
np_result = np.matmul(np_matrix1, np_matrix2)
end = time.time()
print(f"NumPy time: {end - start} seconds")

PyTorch time: 0.01648545265197754 seconds
NumPy time: 0.016958236694335938 seconds


In [18]:
matrix_size = 1000
num_runs = 100

# Create random matrices using NumPy
np_matrix1 = np.random.standard_normal(matrix_size, matrix_size)
np_matrix2 = np.random.standard_normal(matrix_size, matrix_size)

# Convert NumPy matrices to PyTorch tensors
torch_matrix1 = torch.tensor(np_matrix1, dtype=torch.float64)
torch_matrix2 = torch.tensor(np_matrix2, dtype=torch.float64)

# Define functions for matrix multiplication
def numpy_multiplication():
    res = np.matmul(np_matrix1, np_matrix2)
    return res

def torch_multiplication():
    res = torch.matmul(torch_matrix1, torch_matrix2)
    return res

# Time NumPy matrix multiplication using timeit
numpy_time = timeit.timeit(numpy_multiplication, number=num_runs) / num_runs
print(f'Average time for NumPy matrix multiplication: {numpy_time:.6f} seconds')

# Time PyTorch matrix multiplication using timeit
torch_time = timeit.timeit(torch_multiplication, number=num_runs) / num_runs
print(f'Average time for PyTorch matrix multiplication: {torch_time:.6f} seconds')

Average time for NumPy matrix multiplication: 0.007726 seconds
Average time for PyTorch matrix multiplication: 0.004296 seconds


In [23]:
import numpy as np
import torch
import timeit
import time

def sigmoid_where(z: np.ndarray) -> np.ndarray:
    """
    Numerically stable sigmoid function using np.where.

    Parameters:
    z (np.ndarray): Input array.

    Returns:
    np.ndarray: Sigmoid of input array.
    """
    return np.where(
        z >= 0,
        1 / (1 + np.exp(-z)),
        np.exp(z) / (1 + np.exp(z))
    )

def sigmoid_mask(z: np.ndarray) -> np.ndarray:
    """
    Numerically stable sigmoid function using explicit masking.

    Parameters:
    z (np.ndarray): Input array.

    Returns:
    np.ndarray: Sigmoid of input array.
    """
    result = np.zeros_like(z, dtype=np.float64)
    pos_mask = z >= 0
    neg_mask = ~pos_mask
    result[pos_mask] = 1 / (1 + np.exp(-z[pos_mask]))
    result[neg_mask] = np.exp(z[neg_mask]) / (1 + np.exp(z[neg_mask]))
    return result

def sigmoid_torch(z: np.ndarray) -> np.ndarray:
    """
    Numerically stable sigmoid function using PyTorch.

    Parameters:
    z (np.ndarray): Input array.

    Returns:
    np.ndarray: Sigmoid of input array.
    """
    return torch.sigmoid(torch.as_tensor(z)).numpy()

In [32]:
# Define input array
z = np.random.standard_normal(10) * 1000

# Time sigmoid_where using timeit
time_where = timeit.timeit(lambda: sigmoid_where(z), number=1000)
print(f'Average time for sigmoid_where: {time_where:.6f} seconds')

time.sleep(1)

# Time sigmoid_mask using timeit
time_mask = timeit.timeit(lambda: sigmoid_mask(z), number=1000)
print(f'Average time for sigmoid_mask: {time_mask:.6f} seconds')

time.sleep(1)

# Time sigmoid_torch using timeit
time_torch = timeit.timeit(lambda: sigmoid_torch(z), number=1000)
print(f'Average time for sigmoid_torch: {time_torch:.6f} seconds')


/tmp/ipykernel_23633/4031661619.py:16: RuntimeWarning: overflow encountered in exp
  1 / (1 + np.exp(-z)),


Average time for sigmoid_where: 0.008931 seconds
Average time for sigmoid_mask: 0.011018 seconds
Average time for sigmoid_torch: 0.003700 seconds


# USNA : np.dot VS np.linalg.norm

In [ ]:
import numpy as np
import timeit

# Generate random vectors Q and Z of size n
n = 1000  # Example vector size
Q = np.random.standard_normal(n)
Z = np.random.standard_normal(n)

# Define the functions to measure
def dot_product_approach():
    def step(iteration):
        gamma = 1 / iteration
        beta = 1 / (2 * gamma)
        return np.dot(Q, Q) <= beta**2

    for i in range(10_000, 100_000):
        res = step(i)

    return res


def norm_approach():
    def step(iteration):
        gamma = 1 / iteration
        beta = 1 / (2 * gamma)
        return np.linalg.norm(Q) <= beta

    for i in range(10_000, 100_000):
        res = step(i)

    return res


# Measure execution time
dot_time = timeit.timeit(dot_product_approach, number=10)
norm_time = timeit.timeit(norm_approach, number=10)

print("Time using np.dot for squared magnitude: {:.6f} seconds".format(dot_time))
print(
    "Time using np.linalg.norm for squared magnitude: {:.6f} seconds".format(norm_time)
)

# Test if $\Sigma$ is positive definite

In [ ]:
import numpy as np

d = 10
covariance_matrix_article = np.zeros((d, d))

for i in range(d):
    for j in range(d):
        covariance_matrix_article[i, j] = abs(i - j) ** 0.5

eigenvalues_article, eigenvectors_article = np.linalg.eigh(covariance_matrix_article)

print("Eigenvalues article:", eigenvalues_article, "\n")

In [1]:
import random
import numpy as np
import timeit

# Define the number of iterations
iterations = 1000000

# Timing random.randint
time_random = timeit.timeit('random.randint(1, 100)', setup='import random', number=iterations)

# Timing numpy.random.randint
time_np_random = timeit.timeit('np.random.randint(1, 101)', setup='import numpy as np', number=iterations)

print(f"random.randint took: {time_random} seconds for {iterations} iterations")
print(f"np.random.randint took: {time_np_random} seconds for {iterations} iterations")


random.randint took: 0.3966467760001251 seconds for 1000000 iterations
np.random.randint took: 1.445265570000629 seconds for 1000000 iterations


## Test device check speed

In [2]:
import torch
import timeit
from typing import Union

# Function that checks the device inside the function
def concatenate_with_bias_check_inside(tensor: torch.Tensor) -> torch.Tensor:
    bias = torch.tensor([1.0], device=tensor.device)
    return torch.cat((bias, tensor))

# Function that takes the device as an argument
def concatenate_with_bias_with_device(tensor: torch.Tensor, device: Union[str, torch.device]) -> torch.Tensor:
    bias = torch.tensor([1.0], device=device)
    return torch.cat((bias, tensor))

# Example tensors on CPU
tensor_cpu = torch.tensor([1.0, 2.0, 3.0])
device = torch.device('cpu')

# Test for the function that checks the device inside
time_check_inside = timeit.timeit(
    'concatenate_with_bias_check_inside(tensor_cpu)',
    globals=globals(),
    number=1000000
)

# Test for the function that takes the device as an argument
time_with_device = timeit.timeit(
    'concatenate_with_bias_with_device(tensor_cpu, device)',
    globals=globals(),
    number=1000000
)

print(f"Time with device check inside function: {time_check_inside:.6f} seconds")
print(f"Time with device specified as argument: {time_with_device:.6f} seconds")


Time with device check inside function: 2.877647 seconds
Time with device specified as argument: 2.779994 seconds


## torch matmul vs einsum

In [10]:
import torch
import timeit

d = 100
# Create a random matrix and a 1D vector
matrix = torch.randn(d, d)
vector = torch.randn(d)

# Function to perform matrix multiplication using torch.matmul
def matmul_method(matrix, vector):
    return torch.matmul(matrix, vector)

# Function to perform matrix multiplication using torch.einsum
def einsum_method(matrix, vector):
    return torch.einsum('ij,j->i', matrix, vector)

# Test for torch.matmul
time_matmul = timeit.timeit('matmul_method(matrix, vector)', globals=globals(), number=100000)

# Test for torch.einsum
time_einsum = timeit.timeit('einsum_method(matrix, vector)', globals=globals(), number=100000)

print(f"Time for torch.matmul: {time_matmul:.6f} seconds")
print(f"Time for torch.einsum: {time_einsum:.6f} seconds")


Time for torch.matmul: 0.372188 seconds
Time for torch.einsum: 1.164216 seconds


## np.matmul vs np.dot for matrix vector multiplication

In [38]:
import numpy as np
import timeit

# Create a random matrix and a 1D vector
matrix = np.random.standard_normal(100, 100)
vector = np.random.standoard_normal(100)

# Function to perform matrix multiplication using np.matmul
def matmul_method(matrix, vector):
    return np.matmul(vector, matrix)

# Function to perform matrix multiplication using np.dot
def dot_method(matrix, vector):
    return np.dot(matrix.T, vector)

# Test for np.matmul
time_matmul = timeit.timeit('matmul_method(matrix, vector)', globals=globals(), number=10000)

# Test for np.dot
time_dot = timeit.timeit('dot_method(matrix, vector)', globals=globals(), number=10000)

print(f"Time for np.matmul: {time_matmul:.6f} seconds")
print(f"Time for np.dot: {time_dot:.6f} seconds")


Time for np.matmul: 0.579552 seconds
Time for np.dot: 0.340912 seconds


In [68]:
import torch
import numpy
import math
import timeit

# Define a single float
x = 12345.6789

# Function to compute sqrt using torch.sqrt
def torch_sqrt_method(x):
    return torch.sqrt(torch.tensor(x))

# Function to compute sqrt using math.sqrt
def math_sqrt_method(x):
    return math.sqrt(x)

# Function to compute sqrt using numpy.sqrt
def numpy_sqrt_method(x):
    return numpy.sqrt(x)

# Test for torch.sqrt
time_torch_sqrt = timeit.timeit('torch_sqrt_method(x)', globals=globals(), number=1000000)

# Test for math.sqrt
time_math_sqrt = timeit.timeit('math_sqrt_method(x)', globals=globals(), number=1000000)

# Test for numpy.sqrt
time_numpy_sqrt = timeit.timeit('numpy_sqrt_method(x)', globals=globals(), number=1000000)

print(f"Time for torch.sqrt: {time_torch_sqrt:.6f} seconds")
print(f"Time for math.sqrt: {time_math_sqrt:.6f} seconds")
print(f"Time for numpy.sqrt: {time_numpy_sqrt:.6f} seconds")


Time for torch.sqrt: 2.002920 seconds
Time for math.sqrt: 0.038339 seconds
Time for numpy.sqrt: 0.377096 seconds


In [72]:
import torch
import math
import numpy as np
import timeit

# Define a single float
x = 5.6789

# Function to compute exp using torch.exp
def torch_exp_method(x):
    return torch.exp(torch.tensor(x))

# Function to compute exp using math.exp
def math_exp_method(x):
    return math.exp(x)

# Function to compute exp using numpy.exp
def numpy_exp_method(x):
    return np.exp(x)

# Test for torch.exp
time_torch_exp = timeit.timeit('torch_exp_method(x)', globals=globals(), number=1000000)

# Test for math.exp
time_math_exp = timeit.timeit('math_exp_method(x)', globals=globals(), number=1000000)

# Test for numpy.exp
time_numpy_exp = timeit.timeit('numpy_exp_method(x)', globals=globals(), number=1000000)

print(f"Time for torch.exp: {time_torch_exp:.6f} seconds")
print(f"Time for math.exp: {time_math_exp:.6f} seconds")
print(f"Time for numpy.exp: {time_numpy_exp:.6f} seconds")


Time for torch.exp: 2.657747 seconds
Time for math.exp: 0.041092 seconds
Time for numpy.exp: 0.393934 seconds
